In [27]:
import pandas as pd
import os
import requests
import gzip #Combine test and train files
from xml.etree import ElementTree as ET
from collections import defaultdict

In [28]:
os.getcwd()

'C:\\Users\\devsa\\Documents\\Pubtator'

In [10]:
# Combining Gene-Disease test & train data files
# this is from: https://figshare.com/articles/dataset/CoCoScore_Supplementary_Data_v1_0/7198280/1
file1_path = 'dataset_9606_-26_test.tsv.gz'
file2_path = 'dataset_9606_-26_train.tsv.gz'
output_file_path = 'Gene_Disease.tsv.gz'

# Function to combine two TSV files
def combine_tsv_files(file1_path, file2_path, output_file_path):
    with gzip.open(file1_path, 'rb') as file1:
        data1 = file1.read()
    
    with gzip.open(file2_path, 'rb') as file2:
        data2 = file2.read()
    
    combined_data = data1 + data2

    with gzip.open(output_file_path, 'wb') as output_file:
        output_file.write(combined_data)

In [11]:
combine_tsv_files(file1_path, file2_path, output_file_path)
print("Files combined successfully and saved to", output_file_path)

Files combined successfully and saved to Gene_Disease.tsv.gz


In [12]:
# Specify the path to your TSV file
file_path = 'Gene_Disease.tsv.gz'

# Specify column names (replace with your actual column names)
column_names = ['PMID', 'Paragraph No', 'Sentence No', 'Diseas ID', 'Gene ID', 'Text', 'Association_label', 'H']

# Read the tab-separated TSV file into a DataFrame
df = pd.read_csv(file_path, sep='\t', names=column_names)

df['PMID'] = df['PMID'].astype(str)

In [13]:
# Display the first few rows of the DataFrame
print(df.head())

    PMID  Paragraph No  Sentence No   Diseas ID          Gene ID  \
0   1401             2            1    DOID:684  ENSP00000448059   
1  11906             2            1   DOID:2752  ENSP00000305692   
2  15739             2            1  DOID:14749  ENSP00000274813   
3  22474             2            4    DOID:684  ENSP00000357066   
4  36611             2            3  DOID:12800  ENSP00000264914   

                                                Text  Association_label    H  
0  Stationary-phase, minimal deviation UNKDISEASE...                  0   89  
1  We describe an improved method for detecting d...                  1    2  
2  We report a method for rapid prenatal detectio...                  1   52  
3  The UNKGENE (UNKGENE), UNKGENE (UNKGENE), and ...                  0  131  
4  Deficiency of UNKGENE (ARS(B)) is associated w...                  1   75  


In [14]:
len(df)

587208

In [13]:

# Changelog (after Micheal's code) 

# 1.  I merged test and train into one dataset
# 2.  Checked if we can give more that 100 through get method, I tried with 1000 worked for me. 
#     But It takes 3 mins for every 100 request. Did not time how long it took for 1000 - But definitely less than 30 mins.
# 3.  Created a dictionary to store output (#Key: Concept_name, Value: Pubmed_id's) - 
#     created a condition for the PubMed ID to be added to the list of values if it's not already present in the list for 
#     a given concept name.
# 4.  Encapsulated the code to "fetch_pubmed_concepts" function to be used recursively. 
#     Takes your DataFrame "df" and an optional parameter max_pmid_count to specify the maximum number of PMIDs you want to 
#     process. It returns the dictionary as the result.
 
#  - Siva


In [ ]:
#Continued... 31/08/2023

In [31]:
from tqdm import tqdm 

def fetch_pubmed_concepts_df(df, max_pmid_count):
    # Feed DataFrame with a column 'PMID'
    
    pmid_first_n = df['PMID'][:max_pmid_count]
    querylist = pmid_first_n.tolist()
    pubtator_api_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocxml"



    # Query the list as per API states
    for pmid in querylist:
        url = f"{pubtator_api_url}?pmids={pmid}&concepts=gene"
        response = requests.get(url)

        print(f"Submitting request for PMID {pmid}: {url}")

        
        if response.status_code == 200:
            content = response.content
            root = ET.fromstring(content)
            for passage in root.iter('passage'):
                for annotation in passage.iter('annotation'):
                    
                    gene_id = annotation.find("infon[@key='identifier']").text
                    
                    #Gene_name
                    concept_name = annotation.find("text").text.lower()
                    concept_type = annotation.find("infon[@key='type']").text

                    #Couunt occurances per pmid
                    concept_count[(concept_name, pmid)] += 1

                    if concept_count[(concept_name, pmid)] == 1:
                        concept_data.append({
                            'gene_id': gene_id,
                            'concept_name': concept_name,
                            'pubmed_id': pmid,
                            'count': concept_count[(concept_name, pmid)]
                        })
                    else:
                        concept_data[-1]['count'] = concept_count[(concept_name, pmid)]
        else:
            print(f"Request for PMID {pmid} failed with status code {response.status_code}")

    # Create a DataFrame from the concept data
    concept_df = pd.DataFrame(concept_data)

    return concept_df

In [35]:
from tqdm import tqdm #Progress_bar shows status of fetch request from api

def fetch_pubmed_concepts_df(df, max_pmid_count):
     
    # Feed DataFrame with a column 'PMID'    
    pmid_first_n = df['PMID'][:max_pmid_count]
    querylist = pmid_first_n.tolist()
    pubtator_api_url = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocxml"
    
    # List to store concept data as dictionaries
    concept_data = []  
    # To count concept_id occurrences per pubmed_id
    concept_count = defaultdict(int)  

    # Initialize tqdm for the progress bar
    with tqdm(total=len(querylist)) as pbar:
        for pmid in querylist:
            url = f"{pubtator_api_url}?pmids={pmid}&concepts=gene"
            response = requests.get(url)

            if response.status_code == 200:
                content = response.content
                root = ET.fromstring(content)
                for passage in root.iter('passage'):
                    for annotation in passage.iter('annotation'):
                        
                        #NCBI GENE ID (Ref: https://www.ncbi.nlm.nih.gov/gene/)
                        gene_id = annotation.find("infon[@key='identifier']").text 
                        #Gene_name
                        concept_name = annotation.find("text").text.lower()
                        concept_type = annotation.find("infon[@key='type']").text

                        concept_count[(concept_name, pmid)] += 1

                        if concept_count[(concept_name, pmid)] == 1:
                            concept_data.append({
                                'gene_id': gene_id,
                                'concept_name': concept_name,
                                'pubmed_id': pmid,
                                'count': concept_count[(concept_name, pmid)]
                            })
                        else:
                            concept_data[-1]['count'] = concept_count[(concept_name, pmid)]
            else:
                print(f"Request for PMID {pmid} failed with status code {response.status_code}")

            pbar.update(1)  # Update the progress bar

    # Create a DataFrame from the fetched data
    concept_df = pd.DataFrame(concept_data)

    return concept_df

In [37]:
# Usage
# df is the DataFrame, and you can specify the max number of PMIDs you want to process
result_df = fetch_pubmed_concepts_df(df, max_pmid_count=1000)

# Now 'result_df' contains the concept data in a DataFrame with columns 'gene_id', 'concept_name', 'pubmed_id', and 'count'

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [21:40<00:00,  1.30s/it]


In [38]:
print(result_df)

        gene_id                        concept_name pubmed_id  count
0         24616           phenylalanine hydroxylase      1401      4
1          6476                   alpha-glucosidase     11906      1
2          3630                             insulin     22474      1
3           410                     arylsulfatase a     36611      1
4           410                              ars(a)     36611      1
...         ...                                 ...       ...    ...
1329       4790  retinoblastoma gene product (p105)   1673033      1
1330       7984                                 p60   1673033      3
1331  7984;4790                           p60/p105/   1673033      6
1332       7157                                 p53   1673792      8
1333        914                                 cd2   1673843     18

[1334 rows x 4 columns]


In [39]:
csv_file_path = 'result_df.csv'
result_df.to_csv(csv_file_path, index=False)

print(f'Results saved to {csv_file_path}')

Results saved to result_df.csv


In [43]:
# Find rows with semicolon in 'gene_id'
rows_with_semicolon = result_df[result_df['gene_id'].str.contains(';')]

# Display the rows with semicolon in 'gene_id'
rows_with_semicolon

,gene_id,concept_name,pubmed_id,count
66,3073;3074,hex a and b,230195,1
88,718;720,"c3, c4",334183,1
112,3073;3074,hex a- and hex b-),545716,1
115,3105;3106,"hla-a, b, and d",571573,1
147,718;629,"c3, c3, c4, c3-proactivator",919557,1
187,720;718,"c4, c3",1185743,1
198,718;720,"c3, c4",1245600,1
316,4855;18132,notch,1312643,3
429,3479;3481,igf-i and -ii,1339481,2
749,3040;3043,alpha and beta globin,1415186,1


In [58]:
import csv

data = {}
with open('result_df.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    
    for row in reader:
        concept_name = row[1]
        pubmed_id = row[2]
        
        if concept_name not in data:
            data[concept_name] = []
            
        data[concept_name].append(pubmed_id)

In [91]:
print(data)

{'phenylalanine hydroxylase': ['1401', '489556'], 'alpha-glucosidase': ['11906'], 'insulin': ['22474', '182362', '323439', '353918', '739270', '842341', '1105371', '1110865', '1144451', '1147652', '1285360', '1307748', '1333648', '1339481', '1347765', '1377520', '1442714', '1459319', '1550404', '1550446', '1592883', '1641148'], 'arylsulfatase a': ['36611'], 'ars(a)': ['36611'], 'ars(b)': ['36611'], 'mitochondrial aconitase': ['36611'], 'alpha-1': ['48211'], 'alpha-2': ['48211', '71842'], 'antithrombin': ['48766', '578072', '926106'], 'iga': ['52952', '651380', '926106', '1469522'], 'alpha-1-antitrypsin': ['58261', '314560', '1441675', '1542282'], 'alpha1-antitrypsin': ['58261'], 'acid beta-galactosidase': ['62026'], 'albumin': ['65455', '489556', '1498561', '1542282', '1577950'], 'alpha-2-macroglobulin': ['71842'], 'transferrin': ['71842', '1222826', '1498561'], 'haptoglobin': ['71842', '1222826'], 'hla-a': ['77056', '480486', '677553', '1341477', '1349923', '1427838'], 'nerve-growth-f

In [92]:
#Invert the dictionary to map pubmed ids to concepts

pubmed_to_concepts = {}
for concept, pmids in data.items():
    for pmid in pmids:
        if pmid not in pubmed_to_concepts:
             pubmed_to_concepts[pmid] = []
        pubmed_to_concepts[pmid].append(concept)
        

In [93]:
pubmed_to_concepts

{'1401': ['phenylalanine hydroxylase'],
 '489556': ['phenylalanine hydroxylase', 'albumin'],
 '11906': ['alpha-glucosidase'],
 '22474': ['insulin'],
 '182362': ['insulin', 'proinsulin'],
 '323439': ['insulin', 'glucagon'],
 '353918': ['insulin'],
 '739270': ['insulin'],
 '842341': ['insulin', 'glucagon'],
 '1105371': ['insulin', 'glucagon', 'growth hormone', 'gh'],
 '1110865': ['insulin'],
 '1144451': ['insulin'],
 '1147652': ['insulin'],
 '1285360': ['insulin',
  'gastrin-releasing peptide',
  'grp',
  'vasopressin',
  'avp',
  'neuropeptide y',
  'npy',
  'corticotropin releasing hormone',
  'crh',
  'acth'],
 '1307748': ['insulin'],
 '1333648': ['insulin', 'adrenocorticotropic hormone', 'prolactin'],
 '1339481': ['insulin',
  'nerve growth factor',
  'glucagon',
  'igf-ii',
  'igf-i',
  'insulin-like growth factor i',
  'rbp-1',
  'rbp-2',
  'p107',
  'beta-endorphin',
  'igf-i and -ii'],
 '1347765': ['insulin', 'hla-drb1', 'dqa1', 'dqb1', 'drb1'],
 '1377520': ['insulin', 'igf-1', '

In [94]:
# Find co-occurrences of genes in pubmed ids
# For each pmid, find all pairs of concepts

gene_interactions = []
for pmid, concepts in pubmed_to_concepts.items():
    for gene1, gene2 in combinations(concepts, 2):
        if gene1 != gene2:
            gene_interactions.append((gene1, gene2))

In [96]:
#Remove Redundancy
gene_interactions = set(gene_interactions)
#Intercations
with open('gene_interactions.txt', 'w') as f:
    for gene1, gene2 in gene_interactions:
        f.write(gene1 + " interacts with " + gene2 + "\n")

In [101]:
len(gene_interactions)

1831

In [85]:
with open('gene_interactions.txt', 'r') as file:
    content = file.read()

# Display the file content
print(content)

p53 interacts with mck
cystic fibrosis transmembrane conductance regulator interacts with mdr-1
cd71 interacts with cd5
nf1 interacts with ira2
p53 interacts with tumor necrosis factor-alpha
hla-dqa and -dqb interacts with dqb
p53 interacts with epidermal growth factor receptor
interleukin-4 interacts with intercellular adhesion molecule 1
pddr interacts with vdd1
nadh dehydrogenase subunit 6 interacts with nd6
insulin interacts with igf-1
irf-1 interacts with ifn-alpha 4 and -alpha 6
cd2 interacts with cd1
dqb1 interacts with leukocyte antigen (hla) dqa1
cblc interacts with cbld
pax-6 interacts with pax
igf-i interacts with pgr
cathepsin-d interacts with ps2
insulin interacts with glucagon
interleukin-6 interacts with endothelial leukocyte adhesion molecule 1
il-1 alpha interacts with ifn-alpha
hla interacts with hla-dpb1
interleukin-4 interacts with interleukin-2, -3, -4, -6
p53 interacts with gal4
ki-ras interacts with p21
cd4 interacts with interferon-gamma
cbl interacts with methi

In [86]:
# key functionalities of the PubTator pipeline version 2:

# ## PubTator Pipeline V2

# ### Data Loading
# - Loads Gene-Disease dataset from Figshare and combines test/train files
# - Reads data into pandas DataFrame

# ### PubTator API
# - Queries PubTator API to fetch gene annotations for PMIDs
# - API accepts list of PMIDs and concepts like "gene"  
# - Makes requests in loop for each PMID
# - Parses XML response to extract gene info

# Key highlights are:

# - Loading and combining external dataset
# - Interacting with PubTator API to annotate entities
# - Storing annotations in a DataFrame structure
# - Analyzing annotations to find interactions 
# - Outputting results to file
# - Includes improvements like progress tracking, error handling etc.

########################
#  Change log after V1 to V2
#######################


# ### Data Storage
# - Stores gene annotations in a DataFrame
# - Contains columns for gene ID, name, PMID, type
# - Handles multiple annotations per PMID

# ### Post-Processing
# - Finds gene-gene interactions by mapping PMIDs to concepts
# - Outputs interactions to a text file

# ### Visualisation
# - Exploring sutable tools to visualise large network maps (underway...)

# ### Enhancements
# - Added progress bar to track API requests
# - Handles API errors and failures
# - Improved data structures for faster lookups
# - Added file output for analysis

In [110]:
# # Cytoscape!!

# import requests

# # Create a list of edges in Cytoscape JSON format
# edges = [{'data': {'source': source, 'target': target}} for source, target in gene_interaction]

# # Define the Cytoscape REST API URL
# cytoscape_url = 'http://localhost:1234/v1/'

# # Clear the current network in Cytoscape
# requests.delete(cytoscape_url + 'session')

# # Create a new network in Cytoscape
# requests.post(cytoscape_url + 'networks', json=edges)

# # Layout the network (optional)
# requests.post(cytoscape_url + 'apply/layouts/force-directed')

# # Open Cytoscape to view the network

# print("Open Cytoscape to view the network.")

In [124]:
gene_interaction_list = list(gene_interactions)

# Select the first 100 elements
first_200_elements = gene_interaction_list[:200]

# If you want to convert it back to a set (optional)
subset_associations = set(first_200_elements)

In [125]:
import networkx as nx
import plotly.graph_objects as go

# Create a graph
G = nx.Graph()

# Add nodes and edges to the graph
for interaction in subset_associations:
    source, target = interaction
    G.add_node(source)
    G.add_node(target)
    G.add_edge(source, target)

# Create positions for the nodes using a spring layout (you may need to try different layouts)
pos = nx.spring_layout(G)

# Create edge traces
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

# Create node traces
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',  # Set hoverinfo to 'text' to display node names on hover
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = pos[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['text'] += tuple([node])  # Store the node names for hover display

# Create a figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=0)))

# Show the interactive plot
fig.show()
